# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
print(data.shape)
print(data.columns)
data.head()

(4870, 65)
Index(['id', 'ad', 'education', 'ofjobs', 'yearsexp', 'honors', 'volunteer',
       'military', 'empholes', 'occupspecific', 'occupbroad', 'workinschool',
       'email', 'computerskills', 'specialskills', 'firstname', 'sex', 'race',
       'h', 'l', 'call', 'city', 'kind', 'adid', 'fracblack', 'fracwhite',
       'lmedhhinc', 'fracdropout', 'fraccolp', 'linc', 'col', 'expminreq',
       'schoolreq', 'eoe', 'parent_sales', 'parent_emp', 'branch_sales',
       'branch_emp', 'fed', 'fracblack_empzip', 'fracwhite_empzip',
       'lmedhhinc_empzip', 'fracdropout_empzip', 'fraccolp_empzip',
       'linc_empzip', 'manager', 'supervisor', 'secretary', 'offsupport',
       'salesrep', 'retailsales', 'req', 'expreq', 'comreq', 'educreq',
       'compreq', 'orgreq', 'manuf', 'transcom', 'bankreal', 'trade',
       'busservice', 'othservice', 'missind', 'ownership'],
      dtype='object')


,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p><b>Question 1:</b> What test is appropriate for the problem? Does the Central Limit Theorem apply?</p>
<p><b>Answer:</b> We want to determine wether the proportion of callbacks for white-sounding names differs from that for black-sounding names. While the callback data itself is modeled by a binomial distribution for each population (yes or no), the sampling distribution of the callback propotions for each population is normal. As the sample size is large (2435 callbacks per population) and the sampling distribution of the proportions is normal, the Central Limit Theorem does apply. We can calculate the mean and standard deviation of the sampling distribution of the difference in the callback proportions between black-sounding names and white-sounding names and then use them to perform a t-test and determine the p-value.
</p>

<p><b>Question 2:</b> What are the null and alternative hypotheses?</p>
<p><b> Answer: </b></p>
<p><b><i>H<sub>0</sub>:</i></b> The proportion of callbacks for white- and black-sounding names are the same </p>
<p><b><i>H<sub>A</sub>:</i></b> The proportion of callbacks for white- and black-sounding names are different </p>

<p><b>Question 3:</b> Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.</p>
<p><b>Answer:</b> See the code in the following two cells for both a frequentist and bootstrapping approach.</p> 
        
</div>


In [5]:
# Frequentist method for calculating moe, confidence interval, and p-value

# Separate the datasets for black- and white-sounding names
w = data[data.race=='w']
b = data[data.race=='b']

# Calcualte the number of callbacks for each population
b_callbacks = sum(data[data.race=='b'].call)
b_total = data[data.race=='b'].call.count()
w_callbacks = sum(data[data.race=='w'].call)
w_total = data[data.race=='w'].call.count()
print('Out of', b_total, 'black-sounding names,', b_callbacks, 'received callbacks') 
print('Out of', w_total, 'white-sounding names,', w_callbacks, 'received callbacks')

# Calculate the mean of the sampling distribution - the mean is equal to the proportion of callbacks
b_mean = b_callbacks / b_total
w_mean = w_callbacks / w_total
print(round(b_mean*100, 2), '% of black-sounding names and', round(w_mean*100, 2), '% of white-sounding names received callbacks.')

# Calculate the variance of each sampling distribution - var = p(1-p)/n
b_var = (b_mean*(1-b_mean)) / b_total
w_var = (w_mean*(1-w_mean)) / w_total

# Calculate the mean of the distribution of the differences between the two populations, i.e. w_mean - b_mean
diff_mean = w_mean - b_mean
print('Difference in percentage of callbacks:', round(diff_mean*100, 3), '%')

# Calculate the standard error of the distribution of the differences, i.e. the square root of w_var + b_var
diff_se = np.sqrt(w_var + b_var)

#Critical z-score for 95% confidence - as n is large, z-score is appropriate for this test
crit_z = stats.norm.ppf(0.975)

# 95% Margin of error 
moe = crit_z * diff_se
print('The margin of error of the mean difference is', round(moe*100, 3), '%')

# 95% confidence interval
c_int_upper = diff_mean + moe
c_int_lower = diff_mean - moe
print('and the confidence interval is', round(c_int_lower*100, 3), '% to', round(c_int_upper*100, 3), '%')

z_score, p_value = stats.ttest_ind(w['call'], b['call'], equal_var=False)

print('The z-score under the null hypothesis is', round(z_score, 2), 'compared to the critical z-score of',
     round(crit_z, 2), 'for 95% confidence')
print('and the p-value for the null hypothesis is', round(p_value, 7))

Out of 2435 black-sounding names, 157.0 received callbacks
Out of 2435 white-sounding names, 235.0 received callbacks
6.45 % of black-sounding names and 9.65 % of white-sounding names received callbacks.
Difference in percentage of callbacks: 3.203 %
The margin of error of the mean difference is 1.526 %
and the confidence interval is 1.678 % to 4.729 %
The z-score under the null hypothesis is 4.11 compared to the critical z-score of 1.96 for 95% confidence
and the p-value for the null hypothesis is 3.94e-05


In [6]:
# Bootstrapping method for calculating moe, confidence interval, and p-value

# Function to generate permutation sample
def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets
    data = np.concatenate((data1, data2))

    # Permute the concatenated array
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two
    perm_sample_1 = pd.Series(permuted_data[:len(data1)])
    perm_sample_2 = pd.Series(permuted_data[len(data1):])

    return perm_sample_1, perm_sample_2

# Function to generate permutation replicates
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    
    return perm_replicates   

# Calculate difference of proportions
def diff_of_props(data_1, data_2):
    """Difference in proportions of two arrays with binomial data."""
    
    # Proportions of data_1, data_2
    prop_1 = data_1[data_1==1].count() / len(data_1)
    prop_2 = data_2[data_2==1].count() / len(data_2)
    # The difference of proportions of data_1, data_2
    diff = prop_1 -prop_2

    return diff

# Compute difference of mean callback proportion 
diff_props = diff_of_props(w['call'], b['call'])
print('The difference between the callback proportions is', round(diff_props*100, 3), '%')

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w['call'], b['call'], diff_of_props, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= diff_props) / len(perm_replicates)

# Compute the 95% confidence interval of the null hypothesis
c_int = np.percentile(perm_replicates, [2.5, 97.5])

print('The 95% confidence interval of the null hypothesis is', c_int)
print('The p-value for the null hypothesis is', round(p, 4))

The difference between the callback proportions is 3.203 %
The 95% confidence interval of the null hypothesis is [-0.01560575  0.01478439]
The p-value for the null hypothesis is 0.0001


<div class="span5 alert alert-success">
<p><b>Question 4:</b> Write a story describing the statistical significance in the context or the original problem. </p>
<p><b>Answer:</b> Both the frequentist and bootstrapping analysis resulted in a very low p-value for the null hypothesis, which means that we must reject the null hypothesis that there is no difference in the percentage of callbacks between applicants with white-sounding names and black-sounding names. As a result, we must conclude that there is indeed a statistically significant difference between the percentage of callbacks between white-sounding and black-sounding names. This analysis presents strong evidence of racial discrimination in the US labor market, as the dataset was specifically designed to remove other potential factors in callback rate by presenting identical resumes for both groups.</p>
    
<p><b>Question 5:</b> Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</p>
<p><b>Answer:</b> The analysis indicates that race certainly is a factor that effects callback success, but we cannot claim that it is the <i>most important</i> factor in callback success because we have not analyzed the other potential factors. In order to make such a claim, we would need to account for other potential factors (i.e. education, years of experience, previous number of jobs, gender, location, managerial experience, etc) and compare the effects.</p>
</div>